## Keras Tuner

### 1. Setup

In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [2]:
!pip install -q -U keras-tuner
import kerastuner as kt
print('Libraries Imported!')

     |████████████████████████████████| 61kB 3.5MB/s 
Libraries Imported!


### 2. Download and prepare the dataset

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()


4423680/4422102 [==============================] - 0s 0us/step


In [4]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

### 3. Define the model 

In [5]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
  
  return model


### 4. Instantiate the tuner and perform hypertuning

In [6]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'keras_tunar')                       

In [7]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)


In [8]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 384 and the optimal learning rate for the optimizer
is 0.001.



In [9]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4803 - accuracy: 0.8278 - val_loss: 0.4331 - val_accuracy: 0.8423
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3609 - accuracy: 0.8683 - val_loss: 0.4108 - val_accuracy: 0.8477
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3245 - accuracy: 0.8814 - val_loss: 0.3530 - val_accuracy: 0.8719
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3008 - accuracy: 0.8890 - val_loss: 0.3410 - val_accuracy: 0.8801
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2812 - accuracy: 0.8945 - val_loss: 0.3466 - val_accuracy: 0.8758
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2653 - accuracy: 0.9003 - val_loss: 0.3739 - val_accuracy: 0.8665
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2534 - accuracy: 0.9058 - val_loss: 0.3324 - val_accuracy: